In [12]:
import pymodels
from apsuite import lattice_errors
import numpy as np

In [27]:
# Configure errors
dips_error = lattice_errors.DipolesErrors()
quads_error = lattice_errors.QuadsErrors()
quads_skew_error = lattice_errors.QuadsSkewErrors()
sexts_error = lattice_errors.SextsErrors()
girder_error = lattice_errors.GirderErrors()
bpms_error = lattice_errors.BPMErrors()
error_configs = [dips_error, quads_error, sexts_error, quads_skew_error,
                     bpms_error, girder_error]

In [4]:
# Create nominal model and get family data
model = pymodels.si.create_accelerator()
model.cavity_on = False
model.radiation_on = 0
model.vchamber_on = False
famdata = pymodels.si.families.get_family_data(model)

# Create GenerateErrors object
nr_mach = 20
generate_errors = lattice_errors.GenerateErrors()
generate_errors.nr_mach = nr_mach
generate_errors.generate_new_seed()
# generate_errors.seed = 302336
generate_errors.reset_seed()
print(generate_errors.seed)
generate_errors.famdata = famdata
generate_errors.error_configs = error_configs
generate_errors.cutoff = 1
generate_errors.generate_errors(save_errors=True)
errors = generate_errors.load_error_file(
        str(nr_mach) + '_errors_seed_' + str(generate_errors.seed))

527616


In [32]:
def generate_normal_dist(sigma, dim, mean=0.0):
        """Generate errors following a truncated normal distribution.

        Args:
            sigma (float): standart deviation of the errors.
            dim (tuple, list): Tuple with the sizes of each dimension of the
                error distribution. Usually with will  be a tuple in which the
                first element is the number of machines and the second is the
                number of elements where the errors will be applied.
            mean (float, optional): mean value of the errors. Defaults to 0.0.

        Returns:
            numpy.ndarray : array with error distribution with the dimension
                defined by dim.

        """
        idx = np.arange(np.prod(dim))
        dist = np.zeros(idx.size, dtype=float)
        while idx.size:
            dist[idx] = np.random.randn(idx.size)
            idx = (np.abs(dist) > 1).nonzero()[0]
        dist *= sigma
        dist += mean
        return dist.reshape(*dim)

In [35]:
def generate_errors(error_configs, famdata):
        """Generate a dictionary with all errors separated by machine and
             family.

        Args:
            save_errors (bool, optional): If true this dictionary will be
                    saved. Defaults to False.

        Returns:
            dict: dictionary with all errors.

        """
        nr_mach = 2
        fam_errors_dict = dict()
        confs = [(c, f) for c in error_configs for f in c.fam_names]
        for config, fam_name in confs:
            idcs = np.array(
                famdata[fam_name]['index'], dtype="object")
            err_types = {'index': idcs}
            for e_type, sigma in config.sigmas_dict.items():
                if e_type != 'multipoles':
                    err_types[e_type] = generate_normal_dist(
                        sigma=sigma, dim=(nr_mach, len(idcs)))
                    continue
                mpole_dict_n = dict()
                mpole_dict_s = dict()
                for order, mp_value in sigma['normal'].items():
                    mpole_dict_n[order] = generate_normal_dist(
                        sigma=mp_value, dim=(nr_mach, len(idcs)))
                for order, mp_value in sigma['skew'].items():
                    mpole_dict_s[order] = generate_normal_dist(
                        sigma=mp_value, dim=(nr_mach, len(idcs)))
                err_types[e_type] = {
                    'normal': mpole_dict_n, 'skew': mpole_dict_s,
                    'r0': sigma['r0']}
            fam_errors_dict[fam_name] = err_types
     
        return fam_errors_dict

In [37]:
fam_errors_dict = generate_errors(error_configs, famdata)

In [39]:
fam_errors_dict.keys()

dict_keys(['B1', 'B2', 'BC', 'Q1', 'Q2', 'Q3', 'Q4', 'QFA', 'QDA', 'QFB', 'QDB1', 'QDB2', 'QFP', 'QDP1', 'QDP2', 'SFA0', 'SDA0', 'SFA1', 'SDA1', 'SFA2', 'SDA2', 'SDA3', 'SFB0', 'SDB0', 'SFB1', 'SDB1', 'SFB2', 'SDB2', 'SDB3', 'SDP0', 'SFP0', 'SFP1', 'SDP1', 'SFP2', 'SDP2', 'SDP3', 'QS', 'BPM', 'girder'])